# Pytorch로 딥러닝 제대로 배우기-중급
## Part6-3: CNN 실습 (1)
### 목차
1. 데이터 호출(Cifar-10)
2. 모델
3. 학습
4. 모델 변경
 - Kernel Size 변경
 - Stride 변경

### (1) 데이터 호출 (Cifar-10)

In [1]:
import torch
import torch.nn.functional as F
from torch import nn

from torch.utils.data import DataLoader

import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor, Normalize, Compose

In [2]:
## Transform 지정 - ** 초급강좌에서 설명 **
transform = Compose(
    [ToTensor(),
     Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Batch size와 학습의 관계? 
batch_size = 64

trainset = datasets.CIFAR10(root='./dataset', train=True,
                            download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./dataset', train=False,
                           download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


### (2) 모델

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 3)
    self.fc1 = nn.Linear(16 * 6 * 6, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
  
  def forward(self, x):
    # Feature extractor
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = torch.flatten(x, 1)

    # Classifier
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    logits = self.fc3(x)
    return logits


model = Net().to(device)
print(model)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


#### Weights 살펴보기

In [4]:
for k, v in model.state_dict().items():
  print(k)
  print(v.size())

conv1.weight
torch.Size([6, 3, 5, 5])
conv1.bias
torch.Size([6])
conv2.weight
torch.Size([16, 6, 3, 3])
conv2.bias
torch.Size([16])
fc1.weight
torch.Size([120, 576])
fc1.bias
torch.Size([120])
fc2.weight
torch.Size([84, 120])
fc2.bias
torch.Size([84])
fc3.weight
torch.Size([10, 84])
fc3.bias
torch.Size([10])


### (3) 학습

#### 학습/평가 함수 정의

In [5]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [6]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

#### SGD

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

In [8]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(testloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.278451  [    0/50000]
loss: 2.304286  [ 6400/50000]
loss: 2.302286  [12800/50000]
loss: 2.300053  [19200/50000]
loss: 2.291372  [25600/50000]
loss: 2.280086  [32000/50000]
loss: 2.280983  [38400/50000]
loss: 2.276925  [44800/50000]
Test Error: 
 Accuracy: 19.1%, Avg loss: 2.227071 

Epoch 2
-------------------------------
loss: 2.241318  [    0/50000]
loss: 2.233888  [ 6400/50000]
loss: 2.203627  [12800/50000]
loss: 2.040662  [19200/50000]
loss: 2.036546  [25600/50000]
loss: 2.054480  [32000/50000]
loss: 1.837417  [38400/50000]
loss: 1.944051  [44800/50000]
Test Error: 
 Accuracy: 28.6%, Avg loss: 1.953673 

Epoch 3
-------------------------------
loss: 1.983655  [    0/50000]
loss: 1.972011  [ 6400/50000]
loss: 2.051633  [12800/50000]
loss: 1.908737  [19200/50000]
loss: 1.879934  [25600/50000]
loss: 1.703163  [32000/50000]
loss: 1.940786  [38400/50000]
loss: 1.674102  [44800/50000]
Test Error: 
 Accuracy: 36.4%, Avg loss: 1.774809 

Epoc

#### Adam

In [9]:
model = Net().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [10]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(testloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302463  [    0/50000]
loss: 1.873215  [ 6400/50000]
loss: 1.577879  [12800/50000]
loss: 1.899320  [19200/50000]
loss: 1.612406  [25600/50000]
loss: 1.579356  [32000/50000]
loss: 1.585412  [38400/50000]
loss: 1.543841  [44800/50000]
Test Error: 
 Accuracy: 48.5%, Avg loss: 1.413181 

Epoch 2
-------------------------------
loss: 1.465900  [    0/50000]
loss: 1.514005  [ 6400/50000]
loss: 1.279096  [12800/50000]
loss: 1.561292  [19200/50000]
loss: 1.469343  [25600/50000]
loss: 1.484890  [32000/50000]
loss: 1.595582  [38400/50000]
loss: 1.420920  [44800/50000]
Test Error: 
 Accuracy: 51.3%, Avg loss: 1.334411 

Epoch 3
-------------------------------
loss: 1.550385  [    0/50000]
loss: 1.294093  [ 6400/50000]
loss: 1.450516  [12800/50000]
loss: 1.144892  [19200/50000]
loss: 1.326063  [25600/50000]
loss: 1.480365  [32000/50000]
loss: 1.072794  [38400/50000]
loss: 1.170325  [44800/50000]
Test Error: 
 Accuracy: 55.2%, Avg loss: 1.258878 

Epoc

### (4) 모델변경

#### Kernel size 변경

In [11]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=7, stride=1)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 32, 7)

    # Classifier
    self.fc1 = nn.Linear(32 * 3 * 3, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
  
  def forward(self, x):
    # Feature extractor
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = torch.flatten(x, 1)

    # Classifier
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    logits = self.fc3(x)
    return logits


model = Net().to(device)
print(model)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(7, 7), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 32, kernel_size=(7, 7), stride=(1, 1))
  (fc1): Linear(in_features=288, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [12]:
for k, v in model.state_dict().items():
  print(k)
  print(v.size())

conv1.weight
torch.Size([6, 3, 7, 7])
conv1.bias
torch.Size([6])
conv2.weight
torch.Size([32, 6, 7, 7])
conv2.bias
torch.Size([32])
fc1.weight
torch.Size([120, 288])
fc1.bias
torch.Size([120])
fc2.weight
torch.Size([84, 120])
fc2.bias
torch.Size([84])
fc3.weight
torch.Size([10, 84])
fc3.bias
torch.Size([10])


In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [14]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(testloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.300499  [    0/50000]
loss: 1.814999  [ 6400/50000]
loss: 1.582002  [12800/50000]
loss: 1.569574  [19200/50000]
loss: 1.335899  [25600/50000]
loss: 1.664107  [32000/50000]
loss: 1.659217  [38400/50000]
loss: 1.418952  [44800/50000]
Test Error: 
 Accuracy: 46.0%, Avg loss: 1.472495 

Epoch 2
-------------------------------
loss: 1.379654  [    0/50000]
loss: 1.239146  [ 6400/50000]
loss: 1.555190  [12800/50000]
loss: 1.154325  [19200/50000]
loss: 1.672709  [25600/50000]
loss: 1.394652  [32000/50000]
loss: 1.293836  [38400/50000]
loss: 1.223498  [44800/50000]
Test Error: 
 Accuracy: 50.2%, Avg loss: 1.368119 

Epoch 3
-------------------------------
loss: 1.433774  [    0/50000]
loss: 1.637750  [ 6400/50000]
loss: 1.393953  [12800/50000]
loss: 1.202529  [19200/50000]
loss: 1.096585  [25600/50000]
loss: 1.500191  [32000/50000]
loss: 1.041723  [38400/50000]
loss: 1.371895  [44800/50000]
Test Error: 
 Accuracy: 53.4%, Avg loss: 1.285608 

Epoc

#### Stride 변경

In [15]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=2, stride=2)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 32, 3, stride=2)

    # Classifier
    self.fc1 = nn.Linear(32 * 1 * 1, 20)
    self.fc2 = nn.Linear(20, 10)
  
  def forward(self, x):
    # Feature extractor
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = torch.flatten(x, 1)

    # Classifier
    x = F.relu(self.fc1(x))
    logits = self.fc2(x)
    return logits


model = Net().to(device)
print(model)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(2, 2), stride=(2, 2))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 32, kernel_size=(3, 3), stride=(2, 2))
  (fc1): Linear(in_features=32, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=10, bias=True)
)


In [16]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [17]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(testloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.351080  [    0/50000]
loss: 2.188811  [ 6400/50000]
loss: 2.065897  [12800/50000]
loss: 2.064187  [19200/50000]
loss: 2.056169  [25600/50000]
loss: 1.986820  [32000/50000]
loss: 1.897141  [38400/50000]
loss: 1.935973  [44800/50000]
Test Error: 
 Accuracy: 30.8%, Avg loss: 1.868032 

Epoch 2
-------------------------------
loss: 1.895208  [    0/50000]
loss: 1.855376  [ 6400/50000]
loss: 1.850726  [12800/50000]
loss: 1.996070  [19200/50000]
loss: 1.899893  [25600/50000]
loss: 1.894264  [32000/50000]
loss: 1.758712  [38400/50000]
loss: 1.847470  [44800/50000]
Test Error: 
 Accuracy: 34.0%, Avg loss: 1.788560 

Epoch 3
-------------------------------
loss: 1.649878  [    0/50000]
loss: 1.730261  [ 6400/50000]
loss: 1.752233  [12800/50000]
loss: 1.782947  [19200/50000]
loss: 1.894230  [25600/50000]
loss: 1.658606  [32000/50000]
loss: 1.861511  [38400/50000]
loss: 1.715733  [44800/50000]
Test Error: 
 Accuracy: 34.8%, Avg loss: 1.774969 

Epoc